### Readme
1. 舊版

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from flysimpler import Network
from IPython.display import clear_output

# import CRIRELs_plot as cplot
from matplotlib.colors import ListedColormap

%load_ext autoreload
%autoreload 2
print('done')

done


## network

In [4]:
def simulation(b_eo, e_bias, i_bias, delta_e, parameter, duration=400, index_array=(0,0), Path="ph_eibais_delay"):
    '''
    acc : accuracy
    '''
    acc = 2
    b_e1 = round(e_bias, acc)        # bias
    b_e2 = round(e_bias+delta_e, acc)        # bias
    b_i1 = round(i_bias, acc)        # bias
    b_i2 = round(i_bias, acc)        # bias
    
    weight_arr = (30, 60, 30, 30)
    wee, wei, wie, wii =  weight_arr
    
    print(f"path : {Path}")
    print(f"bias : b_e1:{b_e1}, b_i1={b_i1}")
    

    # parameter =============================================================
    thr = -55
    time_stim = 300
    # population 
    neu_list       = ['inp1', 'inp2',  'e1',  'e2',  'i1',  'i2',  'eo', 'buffer']
    neu_c_list     = [  0.5,     0.5,   0.5,   0.5,   0.5,   0.5,   0.5,      0.5]
    neu_tau_list   = [  20,       20,    20,    20,    20,    20,    20,       20]
    neu_rest_list  = [  -70,     -70,   -70,   -70,   -70,   -70,   -70,      -70]
    neu_reset_list = [-50.1,   -50.1, thr-5, thr-5, thr-5, thr-5, thr-5,    -50.1]
    neu_thr_list   = [  -50,     -50,   thr,   thr,   thr,   thr,   thr,      -50]
    neu_ref_list   = [    5,       5,     2,     2,     2,     2,     5,        5]
    
    
    net = Network()
    # ====================== Generate Conf File =================================
    # (Name, N, C, Taum, RestPot, ResetPot, Threshold)
    for i, neu in enumerate(neu_list):
        net.add_neuron(neu, 1, 
                       neu_c_list[i], 
                       neu_tau_list[i], 
                       neu_rest_list[i], 
                       neu_reset_list[i], 
                       neu_thr_list[i], 
                       neu_ref_list[i])
    
    
    # (Name, Tau, RevPot, FreqExt, MeanExtEff, MeanExtCon)
    net.add_receptor('AMPA', 2,   -0, 0, 10.5, 1)     # excitatory
#     net.add_receptor('GABA', 2, -90, 0,    0, 0)     # inhibitatory
    net.add_receptor('GABA', 5, -90, 0,    0, 0)     # inhibitatory

    net.set_neuron_receptor_all('AMPA')
    net.set_neuron_receptor_all('GABA')
    
    net.add_receptor('AMPA', 20, 0, 0, 10.5, 1)      # excitatory#########################################################
    net.set_neuron_receptor('buffer', 'AMPA')

    # connection
    inp_strength = 45
    # ========== inp1 ========================================
    net.add_target('inp1', 'e1', 'AMPA', inp_strength, 1)     # fixed
    net.add_target('inp1', 'i1', 'AMPA', inp_strength, 1)     # fixed

    # ========== inp2 ========================================
    net.add_target('inp2', 'e2', 'AMPA', inp_strength, 1)     # fixed
    net.add_target('inp2', 'i2', 'AMPA', inp_strength, 1)     # fixed

    # ========== e1 ========================================
    net.add_target('e1', 'e2', 'AMPA', wee, 1)
    net.add_target('e1', 'i1', 'AMPA', wie, 1)
    net.add_target('e1', 'i2', 'AMPA',   0, 1) 
    net.add_target('e1', 'eo', 'AMPA',  50, 1)     # fixed

    # ========== e2 ========================================
    net.add_target('e2', 'e1', 'AMPA', wee, 1)
    net.add_target('e2', 'i1', 'AMPA',   0, 1)
    net.add_target('e2', 'i2', 'AMPA', wie, 1) 
    net.add_target('e2', 'eo', 'AMPA',  50, 1)

    # ========== i1 ========================================
    net.add_target('i1', 'e1', 'GABA', wei, 1)
    net.add_target('i1', 'e2', 'GABA',   0, 1)
    net.add_target('i1', 'i2', 'GABA', wii, 1)

    # ========== i2 ========================================
    net.add_target('i2', 'e1', 'GABA',   0, 1)
    net.add_target('i2', 'e2', 'GABA', wei, 1)
    net.add_target('i2', 'i1', 'GABA', wii, 1)
    
    # ========== eo ========================================
    net.add_target('eo', 'buffer', 'AMPA', 100, 1)
    
    #net.plot_network()          #draw connection

    # ====================== Generate Pro File =================================
    Type_Mem = 'ChangeMembraneNoise'
    Type_Freq = 'ChangeExtFreq'
    Type_End = 'EndTrial'
    
    # ========== time (ms) ===================================
    # (Time, Type, Population, GaussMean, GaussSTD)
    # ========== bias ========================================
    net.add_event(0, Type_Mem, 'buffer', 0.1, 0)
    net.add_event(0, Type_Mem, 'eo', b_eo, 0)
#     net.add_event(0, Type_Mem, 'e1', b_e1, 0)
#     net.add_event(0, Type_Mem, 'e2', b_e2, 0)
#     net.add_event(0, Type_Mem, 'i1', b_i1, 0)
#     net.add_event(0, Type_Mem, 'i2', b_i2, 0)

    # ========== phase stimulus ======================================

        
    
    # Time values
    sampling_rate = 10  # Number of samples per second
    num_samples = int(duration * sampling_rate) +1 
    time_list = np.linspace(0, duration, num_samples)
    
    
    # Parameters
    amplitude, period, phase_delay, dc = parameter
    print(f"para : amplitude:{amplitude}, period={period}, phase_delay={phase_delay}, dc={dc}")

    # Generate the sine wave with phase delay
    wave_1 = amplitude * np.sin(2 * np.pi * time_list /period ) + dc
    wave_2 = amplitude * np.sin(2 * np.pi * time_list /period + phase_delay) +dc
    for tt, w1, w2 in zip (time_list, wave_1, wave_2):
        tt = round(tt, 1)
        w1 = round(w1, 4)
        w2 = round(w2, 4)
        net.add_event(tt, Type_Mem, 'e1', w1+b_e1, 0)
        net.add_event(tt, Type_Mem, 'i1', w1+b_i1, 0)
        net.add_event(tt, Type_Mem, 'e2', w2+b_e2, 0)
        net.add_event(tt, Type_Mem, 'i2', w2+b_i2, 0)


    # ========== end ======================================
    net.add_event(duration+1, Type_End)
    # (Filename, Type, Population, *args)
    net.add_output('MemPot.dat'  , 'MemPot', 'AllPopulation')
    net.add_output('SpikeALL.dat', 'Spike', 'AllPopulation')
    net.add_output('FiringRateALL.dat', 'FiringRate', 'AllPopulation', 20, 10)
    
    # ====================== Output File ================================
    net.output('network.conf', 'network.pro')
    # ====================== simulation =================================================================
    #adjust path to your machine
    !/media/alexander/9e1069d5-cd39-4e83-8eff-49b4335462dd/home/ns-cclolab/Documents/ALEX_0605-20230605T073644Z-001/ALEX_0605/v1_logic_discrete/flysim07_21_2.out -conf network.conf -pro network.pro -nmodel LIF
#    


    # ====================== save data =================================
    f_data = pd.read_csv('FiringRateALL.dat', delim_whitespace=True, names=['time']+list(neu_list)).to_numpy()
    g_data = pd.read_csv('MemPot.dat', delim_whitespace=True, names=['time']+list(neu_list)).to_numpy()
    df = pd.DataFrame(f_data, columns=['time']+list(neu_list))
    dg = pd.DataFrame(g_data, columns=['time']+list(neu_list))
    
    
    out_bias_name = f"e_{ii:03d}_i_{jj:03d}_deltat_{delta_e}"
    outnameg = f"V_e_{ii:03d}_i_{jj:03d}"
    df['eo'].to_csv(Path+"/"+ out_bias_name+".txt", index=False, sep='\t')
    dg['eo'].to_csv(Path+"_V/"+ outnameg, index=False, sep='\t')


    
    clear_output() # 清除輸出

# simulation 

## Bias

In [7]:
phase_delay_list = [0,2.*np.pi/20, 1*np.pi/1,-2.*np.pi/20]

b_eo = 0
#e_bias = 0
#i_bias = 0
delta_e = 0


amplitude = .5
period = 500
#phase_delay = 1*np.pi/10 #np.pi/10  # Phase delay (in radians)
dc = 0
duration = 1500



e_bias_list = np.arange(-1, 5, 0.25)
i_bias_list = np.arange(-1, 5, 0.25)

for p, phase_delay in enumerate(phase_delay_list):
    Path = f"ph4_delay_{p:02d}"
    parameter = (amplitude, period, phase_delay, dc)

    for ii, e_bias in enumerate(e_bias_list):
        for jj, i_bias in enumerate(i_bias_list):

            index_array = (ii, jj)
            simulation(b_eo, e_bias, i_bias, delta_e, parameter, duration, index_array, Path)

clear_output() # 清除輸出
print("finish") 

finish
